Подготовка данных

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
path = "/content/drive/MyDrive/diamonds_2.csv"

In [3]:
data  = pd.read_csv(path, sep = ',')

In [4]:
data

,cut,color,clarity,carat,depth,table,price,x,y,z
0,-0.538094,-0.937154,-0.484260,-1.198157,-0.174090,-1.099662,-0.904087,-1.587823,-1.536181,-1.571115
1,0.434945,-0.937154,-1.064107,-1.240350,-1.360726,1.585514,-0.904087,-1.641310,-1.658759,-1.741159
2,-1.511133,-0.937154,0.095588,-1.198157,-3.384987,3.375631,-0.903836,-1.498677,-1.457382,-1.741159
3,0.434945,1.414259,0.675435,-1.071577,0.454129,0.242926,-0.902081,-1.364959,-1.317293,-1.287708
4,-1.511133,2.002112,-0.484260,-1.029384,1.082348,0.242926,-0.901831,-1.240155,-1.212227,-1.117663
...,...,...,...,...,...,...,...,...,...,...
53935,-0.538094,-1.525007,-1.064107,-0.164426,-0.662705,-0.204603,-0.294728,0.016798,0.022304,-0.054887
53936,-1.511133,-1.525007,-1.064107,-0.164426,0.942744,-1.099662,-0.294728,-0.036690,0.013548,0.100987
53937,1.407985,-1.525007,-1.064107,-0.206619,0.733338,1.137985,-0.294728,-0.063434,-0.047740,0.030135
53938,0.434945,0.826406,-0.484260,0.130926,-0.523100,0.242926,-0.294728,0.373380,0.337503,0.285201


In [5]:
y = data['price']

In [6]:
y

,price
0,-0.904087
1,-0.904087
2,-0.903836
3,-0.902081
4,-0.901831
...,...
53935,-0.294728
53936,-0.294728
53937,-0.294728
53938,-0.294728


In [7]:
del data['price']


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.33, random_state=42)

Импорт библиотек

In [10]:
pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00


In [11]:
import onnx
import onnxruntime
import onnxruntime as ort
import tracemalloc
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
import os

Загрузка модели и анализ ее работы

In [12]:
#загрузка модели ONNX и проверка

onnx_model = onnx.load("/content/drive/MyDrive/model.onnx")
onnx.checker.check_model(onnx_model)

In [22]:
graph = onnx_model.graph

print("Inputs:")
for input_tensor in graph.input:
    print(input_tensor)
print("\nOutputs:")
for output_tensor in graph.output:
    print(output_tensor)


Inputs:
name: "input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "unk__6"
      }
      dim {
        dim_value: 9
      }
    }
  }
}


Outputs:
name: "dense_7"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "unk__7"
      }
      dim {
        dim_value: 1
      }
    }
  }
}



In [13]:
onnx_model_path = "/content/drive/MyDrive/model.onnx"
session = ort.InferenceSession(onnx_model_path)

input_name = session.get_inputs()[0].name
x_test_np = x_test.to_numpy().astype('float32')

#отслеживание распределения памяти
tracemalloc.start()
#отслеживание времени работы программы
start_time = time.time()

predictions = session.run(None, {input_name: x_test_np})

end_time = time.time()
current, peak = tracemalloc.get_traced_memory()

print(f"Текущее использование памяти: {current / 10**6} MB")
print(f"Пиковое использование памяти: {peak / 10**6} MB")

tracemalloc.stop()

inference_time = end_time - start_time
print(f"Время инференса: {inference_time:.4f} секунд")

Текущее использование памяти: 0.002779 MB
Пиковое использование памяти: 0.645404 MB
Время инференса: 0.0115 секунд


In [14]:
pred = predictions[0]
y_test = y_test.reset_index(drop=True)
vec = np.array([p[0] for p in pred])

for i in range(len(y_test)):
    print("Предсказанное значение:", pred[i][0], ", правильное значение:", y_test[i], 'разница: ', np.abs(pred[i][0] - y_test[i]))

Выходные данные были обрезаны до нескольких последних строк (5000).
Предсказанное значение: -0.7736627 , правильное значение: -0.7519350883357616 разница:  0.02172759801219981
Предсказанное значение: -0.87598616 , правильное значение: -0.8862897935528539 разница:  0.010303634705045028
Предсказанное значение: -0.3897674 , правильное значение: -0.4173016341103546 разница:  0.02753422573938291
Предсказанное значение: -0.8217786 , правильное значение: -0.8308935438271497 разница:  0.009114948379609467
Предсказанное значение: -0.20409964 , правильное значение: -0.2400837673407945 разница:  0.035984127090588514
Предсказанное значение: -0.80342203 , правильное значение: -0.7782545734995576 разница:  0.02516746028721606
Предсказанное значение: -0.37188613 , правильное значение: -0.4807190602669299 разница:  0.10883292611928586
Предсказанное значение: 3.1555939 , правильное значение: 3.4719662878083075 разница:  0.316372415737995
Предсказанное значение: -0.65737665 , правильное значение: -0.733

In [15]:
СС_tuner = np.corrcoef(vec, y_test)[0][1]
print(f'Коэффициент корреляции с истинными данными: {СС_tuner}')

Коэффициент корреляции с истинными данными: 0.9888344164492132


In [16]:
#просмотр всех операций в модели
for node in onnx_model.graph.node:
    print(f"Operation: {node.op_type}, Inputs: {node.input}, Outputs: {node.output}")

Operation: MatMul, Inputs: ['input', 'sequential/dense/MatMul/ReadVariableOp:0'], Outputs: ['sequential/dense/MatMul:0']
Operation: Add, Inputs: ['sequential/dense/MatMul:0', 'sequential/dense/BiasAdd/ReadVariableOp:0'], Outputs: ['sequential/dense/BiasAdd:0']
Operation: Relu, Inputs: ['sequential/dense/BiasAdd:0'], Outputs: ['sequential/dense/Relu:0']
Operation: MatMul, Inputs: ['sequential/dense/Relu:0', 'sequential/dense_1/MatMul/ReadVariableOp:0'], Outputs: ['sequential/dense_1/MatMul:0']
Operation: Add, Inputs: ['sequential/dense_1/MatMul:0', 'sequential/dense_1/BiasAdd/ReadVariableOp:0'], Outputs: ['sequential/dense_1/BiasAdd:0']
Operation: Relu, Inputs: ['sequential/dense_1/BiasAdd:0'], Outputs: ['sequential/dense_1/Relu:0']
Operation: MatMul, Inputs: ['sequential/dense_1/Relu:0', 'sequential/dense_2/MatMul/ReadVariableOp:0'], Outputs: ['sequential/dense_2/MatMul:0']
Operation: Add, Inputs: ['sequential/dense_2/MatMul:0', 'sequential/dense_2/BiasAdd/ReadVariableOp:0'], Outputs: 

In [17]:
#вычисление MSE и MAE
mse = mean_squared_error(y_test, pred)
mae = mean_absolute_error(y_test, pred)

print(f'Среднеквадратичная ошибка (MSE) на тестовой выборке: {mse}')
print(f'Средняя абсолютная ошибка (MAE) на тестовой выборке: {mae}')

Среднеквадратичная ошибка (MSE) на тестовой выборке: 0.02344520122633953
Средняя абсолютная ошибка (MAE) на тестовой выборке: 0.09205254714436731


In [18]:
onnx_model_path = "/content/drive/MyDrive/model.onnx"
onnx_model_size = os.path.getsize(onnx_model_path)
print(f"Model Size: {onnx_model_size / (1024 * 1024)} MB")

Model Size: 0.01797199249267578 MB
